In [10]:
from PIL import Image, ImageFilter, ImageDraw
import matplotlib.pyplot as plt
from time import time
from skimage.feature import plot_matches
from skimage.transform import pyramid_gaussian
from utils import *
import cv2 as cv


# multi-scale
N_LAYERS = 4
DOWNSCALE = 2

# original_img1 = img1 = Image.open('/home/gaditek/Receipt-Scanner/CustomerDataset/OPTP/Dollmen-Mall-Clifton/11798-638b75538e43b.png')
img1=cv.imread('/home/gaditek/Receipt-Scanner/CustomerDataset/OPTP/Dollmen-Mall-Clifton/11798-638b75538e43b.png',0)
# gray1 = img1.convert('L')
# gray1 = gray1.filter(ImageFilter.GaussianBlur(radius = 3))
gray1 = np.array(img1)
grays1 = list(pyramid_gaussian(gray1, downscale=DOWNSCALE, max_layer=N_LAYERS, multichannel=False))

# original_img2 = img2 = Image.open('/home/gaditek/Receipt-Scanner/ReceiptDataset/OPTP/optp-dollmen-mall-clifton-karachi/4fOzaLl0WJWWZFB18m6g9tlzgreMh67BlKDywAaV.png')
# gray2 = img2.convert('L')
img2=cv.imread('/home/gaditek/Receipt-Scanner/ReceiptDataset/OPTP/optp-dollmen-mall-clifton-karachi/4fOzaLl0WJWWZFB18m6g9tlzgreMh67BlKDywAaV.png',0)
gray2 = np.array(img2)
grays2 = list(pyramid_gaussian(gray2, downscale=DOWNSCALE, max_layer=N_LAYERS, multichannel=False))

scales = [DOWNSCALE**i for i in range(N_LAYERS)]
print('scales: ', scales, '\n')
features_img1 = Image.fromarray(np.copy(img1))
features_img2 = Image.fromarray(np.copy(img2))

kps1 = []
kps2 = []

ds1 = []
ds2 = []

for i in range(N_LAYERS):
    print('pyramid layer: ', i)
    print('scales[i]: ', scales[i])
    scale_coeff1 = (gray1.shape[1]/grays1[i].shape[1], gray1.shape[0]/grays1[i].shape[0])
    scale_coeff2 = (gray2.shape[1]/grays2[i].shape[1], gray2.shape[0]/grays2[i].shape[0])
    
    print('scale_coeff1: ', scale_coeff1)
    print('scale_coeff2: ', scale_coeff2)
    
    print('grays1[i] shape: ', grays1[i].shape)
    print('grays2[i] shape: ', grays2[i].shape)
    
    scale_kp1 = FAST(grays1[i], N=9, threshold=0.1, nms_window=3)
    scale_kp2 = FAST(grays2[i], N=9, threshold=0.1, nms_window=3)
    
    print('kp1: ', len(scale_kp1))
    print('kp2: ', len(scale_kp2))
    
    for keypoint in scale_kp1:
#         features_img1 = cv2.circle(features_img1, tuple(np.round(keypoint*scale_coeff1).astype(np.int32)), int(3*scales[i]), (0,255,0), 1)
        x0 = np.round(keypoint*scale_coeff1)[0]-3*scales[i]
        y0 = np.round(keypoint*scale_coeff1)[1]-3*scales[i]
        x1 = np.round(keypoint*scale_coeff1)[0]+3*scales[i]
        y1 = np.round(keypoint*scale_coeff1)[1]+3*scales[i]
        draw1 = ImageDraw.Draw(features_img1)
        draw1.ellipse([x0,y0,x1,y1], outline='red', width=1)
    for keypoint in scale_kp2:
#         features_img2 = cv2.circle(features_img2, tuple(np.round(keypoint*scale_coeff2).astype(np.int32)), int(3*scales[i]), (0,255,0), 1)
        x0 = np.round(keypoint*scale_coeff2)[0]-3*scales[i]
        y0 = np.round(keypoint*scale_coeff2)[1]-3*scales[i]
        x1 = np.round(keypoint*scale_coeff2)[0]+3*scales[i]
        y1 = np.round(keypoint*scale_coeff2)[1]+3*scales[i]
        draw2 = ImageDraw.Draw(features_img2)
        draw2.ellipse([x0,y0,x1,y1], outline='red', width=1)
        
    plt.figure(figsize=(20,10))
    plt.subplot(1,2,1)
    plt.imshow(grays1[i], cmap='gray')
    plt.subplot(1,2,2)
    plt.imshow(features_img1)

    plt.figure(figsize=(20,10))
    plt.subplot(1,2,1)
    plt.imshow(grays2[i], cmap='gray')
    plt.subplot(1,2,2)
    plt.imshow(features_img2)
    
#     orientations1, orientations2 = None, None
    orientations1 = corner_orientations(grays1[i], scale_kp1)
    orientations2 = corner_orientations(grays2[i], scale_kp2)
    
    if not (len(scale_kp1) == 0 or len(scale_kp2) == 1):
        # avoding errors when no keypoints are computed (happens with small scales - higher pyramid layers)
        kps1.append(np.round(scale_kp1*scale_coeff1).astype(np.int32))
        kps2.append(np.round(scale_kp2*scale_coeff2).astype(np.int32))
        
        d1 = BRIEF(grays1[i], scale_kp1, orientations=orientations1, mode='uniform', n=128)
        d2 = BRIEF(grays2[i], scale_kp2, orientations=orientations2, mode='uniform', n=128)
        ds1.append(d1)
        ds2.append(d2)
    
    plt.show()
    print('-'*50)


plt.figure(figsize=(20,10))
plt.subplot(1,2,1)
plt.imshow(features_img1)
plt.subplot(1,2,2)
plt.imshow(features_img2)
plt.show()



scale_kps1 = np.vstack(kps1)
scale_kps2 = np.vstack(kps2)

scale_ds1 = np.vstack(ds1)
scale_ds2 = np.vstack(ds2)

# Matching is done after computing all keypoints and descriptors to allow for
# the sorting of all descriptors computed on all scales
scale_ms = match(scale_ds1, scale_ds2, cross_check=True, max_distance=np.inf, distance_ratio=0.5)

print('total matches: ', len(scale_ms))

fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(1,1,1)
plot_matches(ax, img1, img2, np.flip(scale_kps1, 1), np.flip(scale_kps2, 1), scale_ms[:50], 
             alignment='horizontal', only_matches=True)

TypeError: pyramid_gaussian() got an unexpected keyword argument 'multichannel'